In [ ]:
!pip -q install google-genai

In [ ]:
#Configurando a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [79]:
#Instalando o Framwork ADK de Agentes do Google
!pip install -q google-adk

from IPython.display import HTML, Markdown

!pip install -q InquirerPy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.6 MB/s eta 0:00:00


In [82]:
import os
from google.colab import userdata
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
from InquirerPy import inquirer as choose

warnings.filterwarnings("ignore")


In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [76]:
##########################################
# --- Agente 1: Buscador de estabelecimento --- #
##########################################
def agente_buscador_categoria(categoria, localizacao_atual):
  buscador = Agent(
      name = 'agente_bucador_remedio',
      model = 'gemini-2.0-flash',
      instruction="""
      Você é um assistente de pesquisa de estabelecimentos. A sua tarefa é usar a ferramenta de buscar do google (google-search)
      conforme a localização atual para fazer pesquisa dos melhores estabelecimentos de acordo com a sua avaliação seguido a ordem
      nome do estabelecimento, localização e numero para contato, link da pagina web para compra on-line se houver e nota de avaliação
      dos estabelecimentos limitando se nos 5 melhores.
      """,
      description='Agente que busca de estabelecimento, numero para contato, link de pagina web e localização',
      tools=[google_search]
  )
  entrada_do_agente_buscador_por_categoria = f"Tópico: {categoria}\nData de hoje: {localizacao_atual}"
  # Executa o agenteentrada
  categoria = call_agent(buscador, entrada_do_agente_buscador_por_categoria)
  return categoria

In [77]:
################################################
# --- Agente 2: Visualizador de Comentário --- #
################################################
def agente_de_comentario(categoria, melhores_locais):
    comentario = Agent(
        name="agente_de_comentario",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de comenteario dos locais encontrados e vai destacar as palavras mais importantes,
        mas não vai trazer as informações de localização.
        """,
        description="Agente que mostra os comentarios e testaca as palavras mais importantes.",
    )

    entrada_do_agente_de_comentario = f"Tópico:{categoria}\nLocal de acordo com a categoria: {melhores_locais}"
    # Executa o agente
    agente_de_comantario_rate = call_agent(comentario, entrada_do_agente_de_comentario)
    return agente_de_comantario_rate

In [91]:
############################################### Main ######################################################################
while True:
  print("📍 Iniciando o Sistema de Buscar por Estabelecimentos 📍")

  # --- Obter o Tópico do Usuário ---
  categoria = input("Por favor, qual o nome por estabelecimento que vamos procurar hoje por categoria : ")
  localizacao_atual = input("Informe Agora o seu endereço atual: ")

  # Inserir lógica do sistema de agentes ################################################
  if not categoria:
    print('Você esqueceu de inserir o da categoria!')
  else:
    print(f'Maravilha! fazer a busca conforme a categoria e lista conforme sua reputação... {categoria}')

    melhores_locais = agente_buscador_categoria(categoria, localizacao_atual)

    print('\n--- 🔎📍 Resultado do Agente 1 (Buscador de remedio) 📍🔎 ---\n')
    display(to_markdown(melhores_locais))
    print('------------------------------------------------------------')

    print('\n--- 💬💬 Você deseja Ver o comentario e endereço desses locais encontrados? 💬💬 ---')
    opcao = input("sim ou nao: ")
    if(opcao == 'sim' ):
      # Agente de comentario
      agente_de_comentario_result = agente_de_comentario(local, melhores_locais)
      display(to_markdown(agente_de_comentario_result))
      print('------------------------------------------------------------')
      input("\nPressione Enter para continuar para outra pesquisa...\n")
    else:
      print("\n❓Gostaria de fazer outra pesquisa❓\n")
      cont = input("sim ou nao: ")
      if(cont.lower() == 'sim' ):
        continue
      else:
        print("\nObrigado 🤝 por nos consultar e volte sempre que precisar...\n")
        break



📍 Iniciando o Sistema de Buscar por Estabelecimentos 📍
Por favor, qual o nome por estabelecimento que vamos procurar hoje por categoria : lanche
Informe Agora o seu endereço atual: avenida japura manaus
Maravilha! fazer a busca conforme a categoria e lista conforme sua reputação... lanche

--- 🔎📍 Resultado do Agente 1 (Buscador de remedio) 📍🔎 ---



> Com prazer! Aqui estão algumas opções de lanches bem avaliadas na Avenida Japurá, em Manaus:
> 
> 1.  **Lanches E Pastelaria Colares:** Localizado na Rua Japurá, 25 - Cachoeirinha. Você pode entrar em contato pelo telefone listado na página do SpotWay.
> 2.  **DÁ PRA COMER HAMBURGUERIA:** Está localizado próximo à Avenida Japurá.
> 3.  **Combinados Lanche:** Localizado na Avenida Laguna, 12A, Planalto.
> 4.  **Jandeico Lanches:** Embora não esteja diretamente na Avenida Japurá, está localizado na Praça das Laranjeiras, na Avenida Nilton Lins - Flores, e é conhecido por seus lanches de qualidade. Telefone para Delivery: 92 99356-6766.
> 5.  **Peixaria Japurá:** Localizado na Rua Ferreira Pena, 805 (esquina com a Avenida Japurá). Para informações e reservas, entre em contato via WhatsApp: (92) 99145-6654.
> 
> Espero que isso ajude você a encontrar um ótimo lanche em Manaus!


------------------------------------------------------------

--- 💬💬 Você deseja Ver o comentario e endereço desses locais encontrados? 💬💬 ---
sim ou nao: sim


> Com certeza! Aqui estão os comentários sobre os lanches na Avenida Japurá, com as palavras mais importantes em **negrito**:
> 
> 1.  **Lanches E Pastelaria Colares:** Localizado na **Rua Japurá**, ideal para quem busca **lanches** e **pastéis** na **Cachoeirinha**.
> 2.  **DÁ PRA COMER HAMBURGUERIA:** Próximo à **Avenida Japurá**, perfeito para os amantes de **hambúrgueres**.
> 3.  **Combinados Lanche:** Na **Avenida Laguna**, oferece **combinações** de **lanches**.
> 4.  **Jandeico Lanches:** Conhecido por seus **lanches de qualidade** na **Praça das Laranjeiras**, próximo à **Avenida Nilton Lins**.
> 5.  **Peixaria Japurá:** Localizado na **Rua Ferreira Pena**, na esquina com a **Avenida Japurá**, especializado em **peixes**.


------------------------------------------------------------

Pressione Enter para continuar para outra pesquisa...

📍 Iniciando o Sistema de Buscar por Estabelecimentos 📍
Por favor, qual o nome por estabelecimento que vamos procurar hoje por categoria : petshop
Informe Agora o seu endereço atual: avenida japura manaus
Maravilha! fazer a busca conforme a categoria e lista conforme sua reputação... petshop

--- 🔎📍 Resultado do Agente 1 (Buscador de remedio) 📍🔎 ---



> Com certeza! Aqui estão algumas opções de pet shops na Avenida Japurá, em Manaus, juntamente com os detalhes que você solicitou:
> 
> 1.  **Super Pet Shop - Praça 14 de Janeiro**
> 
>     *   **Localização:** Avenida Japurá, 1023, Praça 14 de Janeiro, Manaus - AM
>     *   **Telefone:** +55 92 3233-5566
> 
>     Este pet shop está bem localizado na Avenida Japurá e oferece uma variedade de serviços e produtos para animais de estimação.
> 
> Como as avaliações e links para páginas web de compra online podem variar e nem sempre estão disponíveis diretamente, recomendo verificar o site ou entrar em contato diretamente para obter informações mais detalhadas.


------------------------------------------------------------

--- 💬💬 Você deseja Ver o comentario e endereço desses locais encontrados? 💬💬 ---
sim ou nao: nao

❓Gostaria de fazer outra pesquisa❓

sim ou nao: nao

Obrigado 🤝 por nos consultar e volte sempre que precisar...

